In [3]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [4]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200

DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 1000

CALORIES_PER_DAY = 2000
REAL_DATA = True

#### Helper functions

In [5]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [6]:
if not REAL_DATA:
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())

    print(*dishes[:7], sep='n')

#### Real data

In [14]:
if REAL_DATA:
    dishes = {}
    data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
    df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])
    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
    dishes = list(dishes.items())
    print(*dishes[:7], sep='\n')

('Brussels Sprouts Cooked From Frozen Made With Oil', 105.6)
('Dirty Rice', 221.76)
('Wheat Flour White Tortilla Mix Enriched', 449.55)
('Pizza Cheese With Vegetables From Restaurant Or Fast Food Medium Crust', 321.86)
('Cheeseburger On Bun From School', 280.6)
('Fava Beans Canned Drained Fat Added In Cooking', 293.4)
('Soyburger Meatless With Cheese On Bun', 291.2)


#### Sample random menu

In [15]:
def create_random_menu():
    menu = np.ndarray((7, 3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [16]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    return calories


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    return sum(squares)

In [17]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 13452166.036968
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                         │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    604.428 │ [('Restaurant Mexican Spanish Rice', 214.6)                                                                    │
│       │            │  ('Ravioli Cheese-Filled With Tomato Sauce Diet Frozen Meal', 285.6)                                           │
│       │            │  ('Seeds Cottonseed Meal Partially Defatted (Glandless)', 104.228)]                                            │
├───────┼────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    719.478 │ [(

In [18]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [22]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [23]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [24]:
population.sort(key=score_menu)
visualize_menu(population[0])

Score: 1583.420099999988
╒═══════╤════════════╤═════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                      │
╞═══════╪════════════╪═════════════════════════════════════════════════════════════════════════════╡
│     0 │    2010.2  │ [('Dennys Spaghetti And Meatballs', 960.5)                                  │
│       │            │  ('Pork Fresh Shoulder Whole Separable Lean And Fat Cooked Roasted', 394.2) │
│       │            │  ('Stuffed Pot Roast With Potatoes Puerto Rican Style', 655.5)]             │
├───────┼────────────┼─────────────────────────────────────────────────────────────────────────────┤
│     1 │    1986.46 │ [('Tiramisu', 615.96) ('Liver Dumpling', 715.0)                             │
│       │            │  ('Stuffed Pot Roast With Potatoes Puerto Rican Style', 655.5)]             │
├───────┼────────────┼────────────────────────────────────────────